In [3]:
# 패키지 import
import datetime as dt
import pandas as pd
import requests
import time
import urllib.request #
import re     
import konlpy

from bs4 import BeautifulSoup
from pandas import DataFrame
from selenium.webdriver import Chrome
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

In [5]:
#크롬 드라이버 연결
delay=0.1
browser = Chrome()
browser.implicitly_wait(delay)

start_url  = 'https://www.youtube.com/channel/UCtckgmUcpzqGnzcs7xEqMzQ/videos'
browser.get(start_url)  
browser.maximize_window()

body = browser.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출
num_of_pagedowns = 30

#스크롤 다운
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.1)
    num_of_pagedowns -= 1

In [3]:
# 페이지 소스 받아오기
html0 = browser.page_source
html = BeautifulSoup(html0,'html.parser')

In [4]:
# 리스트 선언
title_list = [] #제목 리스트 생성
href_list = [] #주소 리스트 생성
viewcountmake_list = [] #조회수 크롤링 위한 리스트 생성
viewcount_list = [] #조회수 리스트 생성

In [5]:
# 구독자 수 저장
subsc = html.find(id="subscriber-count").text
subsc = subsc[4:]

# title 저장
for tr in html.find_all(id="video-title"):
    title = tr.get('title')
    title_list.append(title)

#href 태그 내용 저장
for tr in html.find_all(id="video-title"):
    href = tr.get('href')
    href_list.append(href)
    
# 조회수 저장
for tr in html.find_all('span', class_="style-scope ytd-grid-video-renderer"):
    viewcount = tr.get_text('span')
    viewcountmake_list.append(viewcount)
    
contents_count = len(title_list)

#구독자수, 동영상 수
print("구독자 수:",subsc, ",영상 개수:", contents_count)

구독자 수: 135만명 ,영상 개수: 133


In [6]:
intro_df = pd.DataFrame({'구분': "펭수", '구독자':subsc, '영상수':contents_count}, index = [1])
intro_df.head()

,구분,구독자,영상수
1,펭수,135만명,133


In [7]:
#전처리 전 title, 주소, 좋아요수 데이터프레임 생성
print(len(title_list), len(href_list), len(viewcountmake_list))
# viewcountmake_list의 경우 조회수와 등록일이 각각 생성되어 list크기가 다름 -> 데이터프레임 생성 불가

133 133 266


In [8]:
# 데이터 클리닝 작업(조회수와 등록일에서 등록일 제거)
for tr in range(0,len(viewcountmake_list),2):
    a = viewcountmake_list[tr]
    viewcount_list.append(a)
    
# 조회수 데이터 전처리
clean_viewcount = []
for i in viewcount_list:
    a = i[4:-2]
    amul = float(a)*10000
    clean_viewcount.append(int(amul))
print(clean_viewcount)

[1220000, 1210000, 630000, 1530000, 670000, 1160000, 1370000, 900000, 1320000, 1030000, 1380000, 1580000, 600000, 1970000, 650000, 1430000, 1420000, 890000, 750000, 1050000, 690000, 330000, 1450000, 580000, 910000, 810000, 730000, 420000, 2280000, 1190000, 180000, 1560000, 200000, 520000, 320000, 810000, 540000, 830000, 430000, 1740000, 490000, 550000, 570000, 350000, 3210000, 310000, 230000, 670000, 350000, 490000, 300000, 550000, 1390000, 100000, 2130000, 1590000, 390000, 850000, 1270000, 800000, 640000, 610000, 370000, 280000, 920000, 510000, 430000, 320000, 640000, 1420000, 300000, 300000, 330000, 93000, 1080000, 370000, 440000, 230000, 1540000, 170000, 290000, 600000, 190000, 230000, 97000, 810000, 1520000, 390000, 840000, 730000, 680000, 100000, 520000, 840000, 360000, 230000, 450000, 480000, 520000, 150000, 130000, 210000, 660000, 210000, 560000, 190000, 100000, 840000, 930000, 1030000, 960000, 230000, 500000, 690000, 700000, 490000, 950000, 220000, 390000, 330000, 480000, 57000

In [9]:
#title, 주소, 좋아요수 데이터프레임 생성
peng_list = pd.DataFrame({'title':title_list, 'href':href_list, 'viewcount':clean_viewcount})

In [10]:
peng_list.head()

,title,href,viewcount
0,[Ep.73] 10살 펭귄 벌써 집 장만,/watch?v=0jQ6W6BPUFc,1190000
1,영화 '천문: 하늘에 묻는다' 오디션 보러 간 펭수(*쿠키영상 있음!),/watch?v=8qYmLVOlYXI,1200000
2,[Ep.72] 수업 중에 경운기를 몰아봤다,/watch?v=OD-Y-CxHz4A,630000
3,[Ep.71] 내 헤드셋 누가 가져갔어?,/watch?v=rRZtVrh6kgI,1520000
4,펭수로 세계 대통합은 이미 시작됐다.,/watch?v=RKi98URQ6GM,670000


In [11]:
peng_list.sort_values(by=['viewcount'], axis=0, ascending=False)

,title,href,viewcount
44,EBS 최초 연습생 펭수의 오디션 합격 TIP *최초공개*,/watch?v=K_5lal40lCk,3200000
28,"[단독] 펭귄 의혹 전격 해부! ㅣ독점 인터뷰: 김민교, 양치승 [Ep.57]",/watch?v=UD-WQvgjsng,2270000
54,"[Ep.44] 펭수, 드디어 ㄱㅁㅈ을 만났다",/watch?v=TckHOovKE2I,2130000
13,[Ep.65] 세상에 나쁜 펭귄은 없다.,/watch?v=wedLGh2jxkQ,1970000
39,EBS 옥상에서 뚝딱이 선배님을 만났다 (feat. 역대급 깜짝손님),/watch?v=yN7wrzXtIWM,1740000
55,[Ep.43]EBS 복지 클라쓰 전격 공개! [힐링 선물 3종세트],/watch?v=80cyxKBcYXA,1580000
11,[Ep.66] 화보 모델 펭수,/watch?v=XUM3sH1kBtw,1580000
31,[Ep.56] '펭TV 야유회'라고 쓰고 '지옥'이라 읽는다,/watch?v=pv84Qcu9DOQ,1550000
78,'자꾸 교육방송 선 넘는' 이말년(침착맨)과 펭수(EBS 연습생)의 짤방 폭격! [...,/watch?v=S9q4BorGg0I,1540000
3,[Ep.71] 내 헤드셋 누가 가져갔어?,/watch?v=rRZtVrh6kgI,1520000


In [12]:
# 영상 본문 key(제목, 조회수, 좋아요, 싫어요, 본문내용, 전체 댓글 수) 추출 - 1

title_num = 0;
n_title_list =[] # 영상별 제목 리스트 생성
story_sub_list = [] # 영상 본문내용 리스트
n_viewcount_list = [] #영상별 조회수 리스트 생성
n_like_list = [] #영상별 좋아요 리스트 생성
n_unlike_list = [] #영상별 싫어요 리스트 생성
n_date_list = [] #영상별 작성일 리스트 생성
n_href_list = [] # 영상별 주소 리스트 생성
start_url  = 'https://www.youtube.com/'

# 데이터
for i in href_list[10:]: 
    url = start_url + i
    browser.get(url)
    print(i)  # 추출 주소값
    n_href_list.append(i)
    
    source = browser.page_source
    bs = BeautifulSoup(source, 'html.parser')
    bsstory = BeautifulSoup(source, "html.parser")
    
    
    # 영상에 대한 기본 정보 수집
    info1 = bs.find('div',{'id':'info-contents'})
    
    #n_title = "" #초기화 계속
    n_title = info1.find('h1',{'class':'title style-scope ytd-video-primary-info-renderer'}).text 
    n_title_list.append(n_title)
    n_viewcount =info1.find('yt-view-count-renderer',{'class':'style-scope ytd-video-primary-info-renderer'}).find_all('span')[0].text  
    n_viewcount_list.append(n_viewcount)
    n_like = info1.find('div',{'id':'top-level-buttons'}).find_all('yt-formatted-string')[0].text #좋아요수
    n_like_list.append(n_like)
    n_unlike = info1.find('div',{'id':'top-level-buttons'}).find_all('yt-formatted-string')[1].text 
    n_unlike_list.append(n_unlike)

    # 영상 본문 내용 추출
    story_sub = bsstory.find('yt-formatted-string', class_='content style-scope ytd-video-secondary-info-renderer').text
    story_sub_list.append(story_sub)
    title_num = title_num + 1; 


/watch?v=yUJAvW2Rykc
/watch?v=XUM3sH1kBtw
/watch?v=LPmyxMH96S8
/watch?v=wedLGh2jxkQ
/watch?v=25RhzK3HuYM
/watch?v=E5f4pRIycsU
/watch?v=2biW2UZBvVA
/watch?v=R2Oro64h5aM
/watch?v=iJF3R-5V2pU
/watch?v=Ja6O9CSzwuk
/watch?v=ThzyHA0LHvI
/watch?v=SlDD1ipgqjs
/watch?v=ze1wbV20qAU
/watch?v=1QNlKu67cwo
/watch?v=5fqCbkIvJuE
/watch?v=sxVKJXb3cp0
/watch?v=5OFDUu8UOvY
/watch?v=KTk8Bu4DRHk
/watch?v=UD-WQvgjsng
/watch?v=Qj2DYNju4Qo
/watch?v=tz_-lCADsbw
/watch?v=pv84Qcu9DOQ
/watch?v=ri5XpIgmoG8
/watch?v=A3_UpfZ1-pE
/watch?v=V4Jh4lNDFfk
/watch?v=AkNQlVM4DrU
/watch?v=WpfexQQkmP4
/watch?v=7SKjzUC77xg
/watch?v=yj_lTuH9qds
/watch?v=yN7wrzXtIWM
/watch?v=33nLNnZrBhE
/watch?v=vlqaa_YxxBE
/watch?v=8eIsC_9hXy0
/watch?v=JgfXjuxnrYM
/watch?v=K_5lal40lCk
/watch?v=zWGdwGfzgnY
/watch?v=I2nxlDdbY0U
/watch?v=S9RTFPo58tI
/watch?v=LwSxAUzH3GE
/watch?v=CJQ5u-RTYys
/watch?v=V5oME0lJBio
/watch?v=x1FfhGkdeiM
/watch?v=QpZHvVyV-wc
/watch?v=VeMfUxAWuuw
/watch?v=TckHOovKE2I
/watch?v=80cyxKBcYXA
/watch?v=9z_LJTXN2Z0
/watch?v=ZH6w

In [14]:
# 조회수 데이터 전처리
clean_n_viewcount = []
for i in n_viewcount_list:
    a = i[4:-1]
    clean_n_viewcount.append(a)

# 좋아요 데이터 전처리
clean_n_like_list = []
for i in n_like_list:
    if i[-1] == '만':
        a = i[:-1]
        amul = float(a)*10000
        clean_n_like_list.append(int(amul))
    elif i[-1] == '천':
        a = i[:-1]
        amul = float(a)*1000
        clean_n_like_list.append(int(amul))
    else:
        clean_n_like_list.append(i)
print(clean_n_like_list)

# 싫어요 데이터 전처리
clean_n_unlike_list = []
for i in n_unlike_list:
    if i[-1] == '천':
        a = i[:-1]
        amul = float(a)*1000
        clean_n_unlike_list.append(int(amul))
    else:
        clean_n_unlike_list.append(i)
#print(clean_n_unlike_list)

[42000, 46000, 29000, 50000, 20000, 38000, 38000, 24000, 27000, 35000, 21000, 13000, 26000, 21000, 21000, 25000, 17000, 14000, 44000, 31000, 6500, 28000, 6300, 13000, 12000, 25000, 13000, 19000, 14000, 36000, 12000, 14000, 13000, 9000, 49000, 7800, 6200, 18000, 10000, 15000, 9700, 13000, 23000, 3400, 35000, 28000, 17000, 14000, 18000, 21000, 18000, 17000, 8400, 8300, 18000, 13000, 9500, 9800, 13000, 24000, 12000, 24000, 7900, 3300, 15000, 7600, 23000, 5600, 22000, 9900, 9300, 11000, 3300, 4400, 2900, 15000, 24000, 10000, 18000, 23000, 15000, 2400, 8600, 13000, 6600, 6900, 12000, 11000, 22000, 3400, 4500, 6700, 21000, 5800, 13000, 5800, 4100, 12000, 18000, 19000, 18000, 7800, 12000, 16000, 12000, 7700, 14000, 5400, 11000, 8800, 11000, 12000, 9600, 5000, 7100, 7100, 11000, 18000, 14000, 23000, 8200, 8800, 12000]


In [27]:
#dataframe 생성

n_peng_list = pd.DataFrame({'제목':n_title_list, '주소':n_href_list, '조회수':clean_n_viewcount,
                           '좋아요':clean_n_like_list, '싫어요':clean_n_unlike_list, '본문내용':story_sub_list})

In [28]:
n_peng_list.describe()

,좋아요,싫어요
count,123.00000,123.000000
mean,15865.04065,170.406504
std,10140.49651,163.160277
min,2400.00000,8.000000
25%,8700.00000,56.500000
50%,13000.00000,124.000000
75%,21000.00000,222.000000
max,50000.00000,839.000000


In [40]:
# 영상별 댓글 추출
title_num = 0;
youtube_pd = pd.DataFrame() # 영상 당 댓글 df 생성용
total_youtube_pd = pd.DataFrame() #최종 df 생성용
str_youtube_comments_len = [] # 영상별 댓글수 리스트 생성

start_url  = 'https://www.youtube.com/'
for i in href_list[10:]: # :2 뺴면 전체 데이터, 시간 단축을 위해 2개 샘플만 시행
    url = start_url + i
    browser.get(url)  
    print(i)  # 추출 주소값
    source = browser.page_source
    bs = BeautifulSoup(source, "html.parser")

    ############################
    # 댓글 추출 코드
    reply_list = [] #댓글 리스트 생성
    body = browser.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출
    num_of_pagedowns = 10

    #스크롤 다운
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        num_of_pagedowns -= 1
    
        html0 = browser.page_source
        html = BeautifulSoup(html0,'html.parser')

        youtube_user_IDs = html.select('div#header-author > a > span')
        youtube_comments = html.select('yt-formatted-string#content-text')
        youtube_likes = html.select('div#toolbar > span')
    
        str_youtube_userIDs = []
        str_youtube_comments = []
        str_youtube_likes = []
        str_youtube_title_number = [] #영상 타이틀 넘버
      
    for i in range(len(youtube_user_IDs)):
        str_tmp = str(youtube_user_IDs[i].text)
        str_tmp = str_tmp.replace('\n', '')
        str_tmp = str_tmp.replace('\t', '')
        str_tmp = str_tmp.replace('               ','')
        str_youtube_userIDs.append(str_tmp)

        str_tmp = str(youtube_comments[i].text)
        str_tmp = str_tmp.replace('\n', '')
        str_tmp = str_tmp.replace('\t', '')
        str_tmp = str_tmp.replace('               ', '')
        str_youtube_comments.append(str_tmp)
    
        str_tmp = str(youtube_likes[i*2].text)
        str_tmp = str_tmp.replace('\n', '')
        str_tmp = str_tmp.replace('\t', '')
        str_tmp = str_tmp.replace('               ','')
        str_youtube_likes.append(str_tmp)
        
        # 어떤 영상인지 넘버링으로 확인
        str_youtube_title_number.append(title_num)
        #print("title_num: ", title_num) #테스트
        
        # 좋아요수 전처리, 좋아요수가 str+공백으로 들어가 있어 공백 제거 후 계산
        clean_str_youtube_likes = []
        for i in str_youtube_likes:
            i = i.replace(" ", '')
            if i.find('만') !=-1:
                a = i.replace('만', '')
                amul = float(a)*10000
                clean_str_youtube_likes.append(int(amul))            
            elif i.find('천') != -1:
                a = i.replace('천', '')
                amul = float(a)*1000
                clean_str_youtube_likes.append(int(amul))
            else:
                clean_str_youtube_likes.append(int(i))
        
         # 리뷰 & 좋아요수 데이터 프레임 생성
        youtube_pd = pd.DataFrame({"ID":str_youtube_userIDs, "Comment":str_youtube_comments, "Like":clean_str_youtube_likes, "Title_number":str_youtube_title_number})
    total_youtube_pd = pd.concat([total_youtube_pd, youtube_pd]) #중복됨
    print(len(youtube_pd))
    
    str_youtube_comments_len.append(len(youtube_pd))
                
    print("")
print("* 본문 추출 개수: ", len(story_sub_list))   #추출한 본문 개수 확인
print("* 댓글 추출 개수: ", len(total_youtube_pd)) #추출한 댓글 개수 확인

/watch?v=yUJAvW2Rykc
40

/watch?v=XUM3sH1kBtw
29

/watch?v=LPmyxMH96S8
40

/watch?v=wedLGh2jxkQ
40

/watch?v=25RhzK3HuYM
40

/watch?v=E5f4pRIycsU
40

/watch?v=2biW2UZBvVA
40

/watch?v=R2Oro64h5aM
40

/watch?v=iJF3R-5V2pU
40

/watch?v=Ja6O9CSzwuk
40

/watch?v=ThzyHA0LHvI
40

/watch?v=SlDD1ipgqjs
40

/watch?v=ze1wbV20qAU
40

/watch?v=1QNlKu67cwo
40

/watch?v=5fqCbkIvJuE
40

/watch?v=sxVKJXb3cp0
40

/watch?v=5OFDUu8UOvY
40

/watch?v=KTk8Bu4DRHk
40

/watch?v=UD-WQvgjsng
40

/watch?v=Qj2DYNju4Qo
40

/watch?v=tz_-lCADsbw
40

/watch?v=pv84Qcu9DOQ
40

/watch?v=ri5XpIgmoG8
40

/watch?v=A3_UpfZ1-pE
40

/watch?v=V4Jh4lNDFfk
40

/watch?v=AkNQlVM4DrU
40

/watch?v=WpfexQQkmP4
40

/watch?v=7SKjzUC77xg
40

/watch?v=yj_lTuH9qds
40

/watch?v=yN7wrzXtIWM
40

/watch?v=33nLNnZrBhE
58

/watch?v=vlqaa_YxxBE
40

/watch?v=8eIsC_9hXy0
40

/watch?v=JgfXjuxnrYM
40

/watch?v=K_5lal40lCk
40

/watch?v=zWGdwGfzgnY
40

/watch?v=I2nxlDdbY0U
40

/watch?v=S9RTFPo58tI
40

/watch?v=LwSxAUzH3GE
40

/watch?v=CJQ5u-RTYys
40



In [2]:
# 영상별 댓글수 추출
start_url  = 'https://www.youtube.com/'
delay=0.1
#browser = Chrome()
#browser.implicitly_wait(delay)
str_n_comments = []

for i in href_list[10:]:#부하 방지용 인덱스
    url = start_url + i
    browser.get(url)
    print(i)  # 추출 주소값
    source = browser.page_source
    bs = BeautifulSoup(source, "html.parser")

    ############################
    # 댓글 추출 코드
    n_comment_list = [] #제목 리스트 생성
    body = browser.find_element_by_tag_name('body')#스크롤하기 위해 소스 추출
    num_of_pagedowns = 5
    
    #스크롤 다운
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        num_of_pagedowns -= 1
    
        html0 = browser.page_source
        html = BeautifulSoup(html0,'html.parser')

        n_comments = html.select('div#title > h2 > yt-formatted-string')  
             
    for i in range(len(n_comments)):
        str_tmp = str(n_comments[i].text)
        str_tmp = str_tmp.replace('댓글 ', '')
        str_tmp = str_tmp.replace('개', '')
        str_n_comments.append(str_tmp) 
print(i, str_n_comments)

NameError: name 'href_list' is not defined

In [33]:
n_peng_list = pd.DataFrame({'제목':n_title_list, '주소':n_href_list, '조회수':clean_n_viewcount,
                           '좋아요':clean_n_like_list, '싫어요':clean_n_unlike_list, '댓글수':str_n_comments, '본문내용':story_sub_list})


#int형 변환
n_peng_list = n_peng_list.astype({'좋아요': int,'싫어요':int})

In [34]:
n_peng_list.head()

,제목,주소,조회수,좋아요,싫어요,댓글수,본문내용
0,[Ep 67] 전 세계 게 섰거라 (feat. 외교부),/watch?v=yUJAvW2Rykc,"1,381,698",42000,749,"6,838",K-펭귄 한다 해외진출
1,[Ep.66] 화보 모델 펭수,/watch?v=XUM3sH1kBtw,"1,583,971",46000,455,"7,678",펭수가 화보 모델로서 촬영을 하게 되었다!\n화보 촬영을 위한 펭수의 눈물(?) 나...
2,펭수와 팬들의 최강 컬래버레이션 신상뮤비 (feat. 챌린지♡),/watch?v=LPmyxMH96S8,"604,376",29000,135,"3,921",힙합펭수의 커버영상이 드. 디. 어. 찾아왔다!\n\n월미도에서 촬영했던 미공개 영...
3,[Ep.65] 세상에 나쁜 펭귄은 없다.,/watch?v=wedLGh2jxkQ,"1,975,593",50000,548,"6,974",어느 날 갑자기 펭수에게 이상한 행동들이 보인다!? \n걱정된 제작진들이 긴급 솔루...
4,수험생은 지금 당장 이 영상을 봅니다 (feat. 정승제쌤),/watch?v=25RhzK3HuYM,"651,078",20000,135,"2,760",내일 모레 엄청 큰 시험을 앞둔\n여러분들을 위해 이 영상을 바칩니당 \n펭펭


In [42]:
print(str_youtube_comments_len)

[20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20, 20]


In [43]:
#추출한 본문 예시 1개 확인
print("예시 1개", print(story_sub_list[0]))

K-펭귄 한다 해외진출
예시 1개 None


In [36]:
# 본문내용 데이터 전처리
clean_story_sub_list = []
for i in story_sub_list:
    a = i.replace('\n', '')
    # 첨부 음악 리스트가 본문내용에 포함된 경우 찾아서 삭제
    delresult = a.find("[MUSIC INFO]")
    delresult1 = a.find("음원 정보")
    delresult2 = a.find("1.")
    if delresult != -1:
        a = a[:delresult]
        clean_story_sub_list.append(a)
    elif delresult1 != -1:
        a = a[:delresult1]
        clean_story_sub_list.append(a)
    elif delresult2 != -1:
        a = a[:delresult2]
        clean_story_sub_list.append(a)
    else:
        clean_story_sub_list.append(a)
print(clean_story_sub_list)

['K-펭귄 한다 해외진출', '펭수가 화보 모델로서 촬영을 하게 되었다!화보 촬영을 위한 펭수의 눈물(?) 나는 노력!그리고 현직 모델에게 직접 받는 모델 포즈까지!', '힙합펭수의 커버영상이 드. 디. 어. 찾아왔다!월미도에서 촬영했던 미공개 영상과 여러분이 함께 만들어준 이번 신상뮤비 공개챌린지에 참여해준 모든 분들 펭러뷰~♡잊지말자 월미도 우리가 함께한 시간여러분이 저를 위해 함께해주는 시간 모두', "어느 날 갑자기 펭수에게 이상한 행동들이 보인다!? 걱정된 제작진들이 긴급 솔루션 대책 마련에 나섰다!세상에 나쁜 펭귄은 없다!========================= 칠판옆 '꿈을 이루기 위한 7가지 충고 - ' 저자 김효석'=========================", '내일 모레 엄청 큰 시험을 앞둔여러분들을 위해 이 영상을 바칩니당 펭펭', '펭수가 똑이 범이 둘리 선배님과 동창회 겸 단풍놀이에 떠납니다.남극 친구들과의 동창회에서 떠나는 즐거운 시간들 시잡합니다', 'SBS에 내레이션 하러 갔다가윤도현 선배님의 비타민 됐음(뿌듯)', '매니저에 적응 중인 2기 매니저스펭수 사관학교를 통해 진정한 매니저로 거듭나보자펭! 하!', '펭수에게도 핑크빛이?! (feat.펭수 열애설)', '자랑하고 싶어서 입이 근질근질했는데이제 대놓고 자랑해도 돼?', '고3들아 기다려라! 펭수가 간다!', '이거 보고 내일 61화 보면 더 꿀잼', '펭수를 S 본부로 초대한 셀럽의 정체가 드러났다..!S 본부의 특급 셀럽 배성재가 직접 펭수의 자질을 확인하고 모종의 거래까지..?', '우리 친구들을 위해일요일 밤에 올립니댯 펭펭', 'SBS의 셀럽을 자초한 누군가가 초대장을 보내왔다!펭수의 SBS 정복기 1편 (feat. 재재)', '펭수는 구독자 여러분들이 있어서하나도 외롭지 않습니다!', '펭수가 드디어 해외진출을?!???펭수 친구 엘로디의 프랑스 진출 꿀팁 대방출!!펭수는 프랑스 진출에 성공할 수 있을까??', '이 영광을구독자 여러분들께 돌립니당', '펭

In [43]:
# title, 주소, 조회수, 본문내용 추가한 데이터 프레임
peng_list_update = pd.DataFrame({'제목':n_title_list, '주소':n_href_list, '조회수':clean_n_viewcount,
                                '좋아요':clean_n_like_list, '싫어요':clean_n_unlike_list, '댓글수':str_n_comments, '본문내용':clean_story_sub_list})

#int형 변환
peng_list_update = peng_list_update.astype({'좋아요': int,'싫어요':int})
peng_list_update.head()

,제목,주소,조회수,좋아요,싫어요,댓글수,본문내용
0,[Ep 67] 전 세계 게 섰거라 (feat. 외교부),/watch?v=yUJAvW2Rykc,"1,381,698",42000,749,"6,838",K-펭귄 한다 해외진출
1,[Ep.66] 화보 모델 펭수,/watch?v=XUM3sH1kBtw,"1,583,971",46000,455,"7,678",펭수가 화보 모델로서 촬영을 하게 되었다!화보 촬영을 위한 펭수의 눈물(?) 나는 ...
2,펭수와 팬들의 최강 컬래버레이션 신상뮤비 (feat. 챌린지♡),/watch?v=LPmyxMH96S8,"604,376",29000,135,"3,921",힙합펭수의 커버영상이 드. 디. 어. 찾아왔다!월미도에서 촬영했던 미공개 영상과 여...
3,[Ep.65] 세상에 나쁜 펭귄은 없다.,/watch?v=wedLGh2jxkQ,"1,975,593",50000,548,"6,974",어느 날 갑자기 펭수에게 이상한 행동들이 보인다!? 걱정된 제작진들이 긴급 솔루션 ...
4,수험생은 지금 당장 이 영상을 봅니다 (feat. 정승제쌤),/watch?v=25RhzK3HuYM,"651,078",20000,135,"2,760",내일 모레 엄청 큰 시험을 앞둔여러분들을 위해 이 영상을 바칩니당 펭펭


In [44]:
# 0번째 영상에서 추출한 댓글 DataFrame
youtube_pd.head()

,ID,Comment,Like,Title_number
0,옷 읽어주는남자,처음부터 애들용이 아니였자나 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,4900,0
1,뿌꾸꾸꾸,펭수 이때 진짜 애기네ㅋㅋㅋㅋㅋㅋ하여튼 애들 크는 건 정말 금방금방이라니까,3600,0
2,Joey CHOI,이 땐 말을 안했어 ㅋㅋㅋㅋㅋㅋㅋ,3600,0
3,다 나가주세요,결국 여기까지 왔다..,1400,0
4,장썬타민,이때부터 보이는 펭성...,1100,0


In [45]:
# 전체 영상에서 추출한 댓글 DataFrame
total_youtube_pd

,ID,Comment,Like,Title_number
0,EBSDocumentary (EBS 다큐),"펭수...곧 백만이네,,, 짜싁 멋지다,,,,날씨가 많이 춥다,,,, 감기 조심허구...",2500,0
1,펭랑단1호,악플러들 고소하고 EBS 빚 갚자,1300,0
2,Isabel la,아니 무슨 교육방송이 ㅋㅋㅋ 왠만한 예능프로그램보다 더 잘만들어 자막이고 편집이고 ...,1000,0
3,월드곰탱이,초반 영상부터 다 봐온 사람으로서열심히 한 죄밖에 없는데너무 힘들게 하는듯나쁜 댓글...,925,0
4,Zaya Kim,펭수야 나는 덴마크 사는 한국사람이야 그런데 여기 친구들한테 너의 동영상을 전도하고...,575,0
5,대한민국외교부,펭수 와줘서 고마워요! 해외진출의 꿈을 응원할게요~ 2019 한·아세안 특별정상회의...,3700,0
6,Bak Shin,"외교부편이 진짜 레전드네요. 어른이들과의 합도 좋고. 꿀벌펭수, 판다펭수, 취권펭수...",446,0
7,gravity,위험한 물품 가지고 있어여? “제 자신” ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,655,0
8,열공열공,외교뷰 10살한테 다산의 상징이라뇨 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,321,0
9,Fine K,회사사람들이 펭수 얘기하길래 탈쓰고 나오는건데 뭐가 재밌나 하고 봤는데 겁나 재밌네...,376,0


In [53]:
# 펭수 intro csv 저장
intro_df.to_csv("peng_intro.csv", mode='w', encoding="utf-8-sig")

# 펭수 동영상 csv 저장
peng_list_update.to_csv("peng.csv", mode='w', encoding="utf-8-sig")

# 펭수 댓글 csv 저장
total_youtube_pd.to_csv("peng_reply.csv", mode='w', encoding="utf-8-sig")